In [1]:
with open("api_key.txt", 'r') as f:
    api_key = f.read()

In [2]:
import pandas as pd

df = pd.read_csv("df.csv")

In [3]:
test_meta = df.iloc[36]['TopicMetadata']
test_solution = df.iloc[36]['Solution']
print(test_meta, test_solution)

(Divisibility)->(The greatest common divisor and the least common multiple) Solution. Let  

$$
a=g c d(a_{1},a_{2},\ldots,a_{n})\quad{\mathrm{and}}\quad b=g c d\left({\frac{a_{1}+a_{2}}{2}},{\frac{a_{2}+a_{3}}{2}},\ldots,{\frac{a_{n}+a_{1}}{2}}\right)
$$  

Then $a_{k}=\alpha_{k}a$ , for some integers $\alpha_{k}$ , $k=1,2,\ldots,n$ . It follows that  

$$
\frac{a_{k}+a_{k+1}}{2}=\frac{\alpha_{k}+\alpha_{k+1}}{2}a,
$$  

where $u_{n+1}=u_{1}$ and $\alpha_{n+1}=\alpha_{1}$ . Since $a_{k}$ are odd numbers, $\alpha_{k}$ are also odd, so $\frac{\alpha_{k}+\alpha_{k+1}}{2}$ are integers.  

From relation (1) it follows that $a$ divides ak + ak+1 for all so a divides b. On the other hand, ak + ak+1 = βkb, for some integers βk. Then  

$$
a_{k}+a_{k+1}\equiv0{\pmod{2b}}
$$  

for all $k\in\{1,2,\ldots,n\}$ . Summing up from $k=1$ to $k=n$ yields  

$$
2(a_{1}+a_{2}+\cdot\cdot\cdot+a_{n})\equiv0{\pmod{2b}},
$$  

hence  

$$
a_{1}+a_{2}+\cdot\cdot\cdot+a_{n}\equiv0{\pmod{b}}.
$$  

Summing up

In [4]:
import semchunk
import tiktoken                        # `transformers` and `tiktoken` are not required.
from transformers import AutoTokenizer # They're just here for demonstration purposes.

chunk_size = 4 # A low chunk size is used here for demonstration purposes. Keep in mind, `semchunk`
               # does not know how many special tokens, if any, your tokenizer adds to every input,
               # so you may want to deduct the number of special tokens added from your chunk size.
text = 'The quick brown fox jumps over the lazy dog.'

# You can construct a chunker with `semchunk.chunkerify()` by passing the name of an OpenAI model,
# OpenAI `tiktoken` encoding or Hugging Face model, or a custom tokenizer that has an `encode()`
# method (like a `tiktoken`, `transformers` or `tokenizers` tokenizer) or a custom token counting
# function that takes a text and returns the number of tokens in it.
chunker = semchunk.chunkerify('deepseek-ai/DeepSeek-V3-0324', chunk_size)

# You can also pass an `offsets` argument to return the offsets of chunks, as well as an `overlap`
# argument to overlap chunks by a ratio (if < 1) or an absolute number of tokens (if >= 1).
chunks, offsets = chunker(text, offsets = True, overlap = 0.5)
print(chunks)
print()
print(offsets)

c:\Users\mokrota\Documents\GitHub\math_problem_recommender\math_problem_recommender\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['The quick brown', 'brown fox jumps', 'fox jumps over the', 'over the lazy', 'lazy dog.']

[(0, 15), (10, 25), (16, 34), (26, 39), (35, 44)]


In [26]:
chunk_size = 100
chunker = semchunk.chunkerify('deepseek-ai/DeepSeek-V3-0324', chunk_size)

def chunk_text(text):
    chunks = chunker(text, overlap=0.4)
    return chunks

chunks = chunk_text(test_solution)
print(test_solution)
print()
print(chunks)

Solution. Let  

$$
a=g c d(a_{1},a_{2},\ldots,a_{n})\quad{\mathrm{and}}\quad b=g c d\left({\frac{a_{1}+a_{2}}{2}},{\frac{a_{2}+a_{3}}{2}},\ldots,{\frac{a_{n}+a_{1}}{2}}\right)
$$  

Then $a_{k}=\alpha_{k}a$ , for some integers $\alpha_{k}$ , $k=1,2,\ldots,n$ . It follows that  

$$
\frac{a_{k}+a_{k+1}}{2}=\frac{\alpha_{k}+\alpha_{k+1}}{2}a,
$$  

where $u_{n+1}=u_{1}$ and $\alpha_{n+1}=\alpha_{1}$ . Since $a_{k}$ are odd numbers, $\alpha_{k}$ are also odd, so $\frac{\alpha_{k}+\alpha_{k+1}}{2}$ are integers.  

From relation (1) it follows that $a$ divides ak + ak+1 for all so a divides b. On the other hand, ak + ak+1 = βkb, for some integers βk. Then  

$$
a_{k}+a_{k+1}\equiv0{\pmod{2b}}
$$  

for all $k\in\{1,2,\ldots,n\}$ . Summing up from $k=1$ to $k=n$ yields  

$$
2(a_{1}+a_{2}+\cdot\cdot\cdot+a_{n})\equiv0{\pmod{2b}},
$$  

hence  

$$
a_{1}+a_{2}+\cdot\cdot\cdot+a_{n}\equiv0{\pmod{b}}.
$$  

Summing up for $k=1,3,\dotsc,n-2$ implies  

$$
a_{1}+a_{2}+\cdot\cdot\cdot+a_{n-1}\eq

In [6]:
prompt = """I will give you topic described hierarchicaly and a chunked solution of problem. You need to highlight chunk (or chunks) that use the bottommost topic in hieararchy (the most specific). As output give me python list of chunks indices without explanations.

TOPIC: {topic}

SOLUTION CHUNKS: {solution_chunks}"""

In [39]:
df.columns

Index(['Unnamed: 0', 'TopicMetadata', 'Problem', 'Solution',
       'Problem&Solution', 'Problem Book No'],
      dtype='object')

In [ ]:
from groq import Groq
import re
import ast
client = Groq(api_key=api_key)

def related_chunks(row):
    """Returns chunks of solution that contain strategy."""

    topic = row['TopicMetadata']
    solution = row['Solution']
    if solution is pd.NA or topic is pd.NA or topic == "(Miscellaneous Problems)":
        return pd.NA
    chunks = chunk_text(solution)

    completion = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {
                "role": "user",
                "content": prompt.format(topic=topic, solution_chunks=chunks)
            }
        ]
    )

    chunk_pattern = r"\[(\s*\d+\s*,?)*\]"
    match = re.search(chunk_pattern, completion.choices[0].message.content.split('</think>')[-1])
    related_chunks_ids = None
    if match:
        related_chunks_ids = ast.literal_eval(match.group())
    rel_chunks = []
    for i in related_chunks_ids:
        try:
            rel_chunks.append(chunks[i])
        except Exception as e:
            print(f"Warning: indice {i} out of range {len(chunks)}")
    return rel_chunks, chunks

In [45]:
df

,Unnamed: 0,TopicMetadata,Problem,Solution,Problem&Solution,Problem Book No
0,0,(Divisibility)->(Divisibility),Problem 1.1.1. Prove that for all integers $n$...,Solution. a) $n^{5}-5n^{3}+4n=n(n^{2}-1)(n^{2}...,Problem 1.1.1. Prove that for all integers $n$...,1.1.1.
1,1,(Divisibility)->(Divisibility),Problem 1.1.2. Let $p>2$ be an odd number and ...,Solution. Define $k=p^{n}$ and note that $k$ i...,Problem 1.1.2. Let $p>2$ be an odd number and ...,1.1.2.
2,2,(Divisibility)->(Divisibility),Problem 1.1.3. Prove that \n\n$$\n3^{4^{5}}+4...,Solution. The given number is of the form $m^{...,Problem 1.1.3. Prove that \n\n$$\n3^{4^{5}}+4...,1.1.3.
3,3,(Divisibility)->(Divisibility),Problem 1.1.4. Find all positive integers $n$ ...,Solution. Let $a$ be the greatest odd integer ...,Problem 1.1.4. Find all positive integers $n$ ...,1.1.4.
4,4,(Divisibility)->(Divisibility),Problem 1.1.5. Find the elements of the set \...,Solution. Since $\frac{x^{3}-3x+2}{2x+1}\in\ma...,Problem 1.1.5. Find the elements of the set \...,1.1.5.
...,...,...,...,...,...,...
641,641,(Miscellaneous Problems),Problem 11.9. Prove that every positive ration...,"Solution. We firstly claim if $m,n$ are positi...",Problem 11.9. Prove that every positive ration...,11.9.
642,642,(Miscellaneous Problems),Problem 11.10. Two positive integers are writt...,Solution. Call the original numbers $x$ and $y...,Problem 11.10. Two positive integers are writt...,11.10.
643,643,(Miscellaneous Problems),Problem 11.11. Let $f(x)+a_{0}+a_{1}x+\cdot\cd...,"Solution. Consider any integers $c_{1},c_{2}$ ...",Problem 11.11. Let $f(x)+a_{0}+a_{1}x+\cdot\cd...,11.11.
644,644,(Miscellaneous Problems),"Problem 11.12. Let $x,a,b$ be positive integer...","Solution. If $x=1$ , then $a=b=1$ and we are d...","Problem 11.12. Let $x,a,b$ be positive integer...",11.12.


In [42]:
test_df = df.head(10)

In [44]:
test_df

,Unnamed: 0,TopicMetadata,Problem,Solution,Problem&Solution,Problem Book No
0,0,(Divisibility)->(Divisibility),Problem 1.1.1. Prove that for all integers $n$...,Solution. a) $n^{5}-5n^{3}+4n=n(n^{2}-1)(n^{2}...,Problem 1.1.1. Prove that for all integers $n$...,1.1.1.
1,1,(Divisibility)->(Divisibility),Problem 1.1.2. Let $p>2$ be an odd number and ...,Solution. Define $k=p^{n}$ and note that $k$ i...,Problem 1.1.2. Let $p>2$ be an odd number and ...,1.1.2.
2,2,(Divisibility)->(Divisibility),Problem 1.1.3. Prove that \n\n$$\n3^{4^{5}}+4...,Solution. The given number is of the form $m^{...,Problem 1.1.3. Prove that \n\n$$\n3^{4^{5}}+4...,1.1.3.
3,3,(Divisibility)->(Divisibility),Problem 1.1.4. Find all positive integers $n$ ...,Solution. Let $a$ be the greatest odd integer ...,Problem 1.1.4. Find all positive integers $n$ ...,1.1.4.
4,4,(Divisibility)->(Divisibility),Problem 1.1.5. Find the elements of the set \...,Solution. Since $\frac{x^{3}-3x+2}{2x+1}\in\ma...,Problem 1.1.5. Find the elements of the set \...,1.1.5.
5,5,(Divisibility)->(Divisibility),Problem 1.1.6. Find all positive integers $n$ ...,Solution. Let $b$ be the last digit of the num...,Problem 1.1.6. Find all positive integers $n$ ...,1.1.6.
6,6,(Divisibility)->(Divisibility),Problem 1.1.7. Find the greatest positive inte...,Solution. The number 23 is prime and divides e...,Problem 1.1.7. Find the greatest positive inte...,1.1.7.
7,7,(Divisibility)->(Divisibility),Problem 1.1.8. Find the positive integers n wi...,"Solution. Of course, there is $1\leq i\leq12$ ...",Problem 1.1.8. Find the positive integers n wi...,1.1.8.
8,8,(Divisibility)->(Divisibility),Problem 1.1.9. Let n be a positive integer. Sh...,"Solution. Suppose, on the contrary, that there...",Problem 1.1.9. Let n be a positive integer. Sh...,1.1.9.
9,9,(Divisibility)->(Divisibility),Problem 1.1.10. Show that for any natural numb...,NaN,Problem 1.1.10. Show that for any natural numb...,1.1.10.


In [43]:
test_df[["Chunks", "RelatedChunks"]] = test_df.apply(related_chunks, axis=1)

TypeError: 'float' object is not iterable

In [ ]:

for topic, solution in zip(topics, solutions):
